In [1]:
import csv
import glob
import os
import time

import pandas as pd
from cassandra.cluster import Cluster

import players_by_song
import songs_by_player
import songs_by_session

keyspace = "udacity"
final_csv = 'event_datafile_new.csv'

In [2]:
def create_filepaths():
    """
    copied from template notebook: create a list of csv files to loop through
    """
    filepath = os.getcwd() + '/event_data'

    for root, dirs, files in os.walk(filepath):
        file_path_list = glob.glob(os.path.join(root, '*'))
    return file_path_list

In [3]:
def create_final_csv(file_path_list):
    """
    copied from template notebook: create smaller, final CSV for inserting rows
    """
    full_data_rows_list = []

    for f in file_path_list:
        with open(f, 'r', encoding='utf8', newline='') as csvfile:
            csvreader = csv.reader(csvfile)
            next(csvreader)
            for line in csvreader:
                full_data_rows_list.append(line)

    csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

    with open(final_csv, 'w', encoding='utf8', newline='') as f:
        writer = csv.writer(f, dialect='myDialect')
        writer.writerow(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
                         'level', 'location', 'sessionId', 'song', 'userId'])
        for row in full_data_rows_list:
            if row[0] == '':
                continue
            writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

    with open('event_datafile_new.csv', 'r', encoding='utf8') as f:
        final_count = sum(1 for line in f)

    assert final_count == 6821

In [4]:
def recreate_keyspace(session):
    """
    create keyspace, if necessary
    """
    create_keyspace_command = f"CREATE KEYSPACE IF NOT EXISTS {keyspace} WITH REPLICATION = "
    create_keyspace_command += "{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"
    session.execute(create_keyspace_command)

In [5]:
def use_keyspace(session):
    """
    set the session keyspace
    """
    session.set_keyspace(keyspace)

In [6]:
def drop_tables(session):
    """
    drop tables, if they exist
    """
    start = time.perf_counter()
    session.execute(songs_by_session.drop)
    session.execute(songs_by_player.drop)
    session.execute(players_by_song.drop)
    stop = time.perf_counter()
    print(f"drop_tables in {stop - start:0.4f} seconds")

In [7]:
def create_tables(session):
    """
    (re)create tables
    """
    start = time.perf_counter()
    session.execute(songs_by_session.create)
    session.execute(songs_by_player.create)
    session.execute(players_by_song.create)
    stop = time.perf_counter()
    print(f"create_tables in {stop - start:0.4f} seconds")

In [8]:
def populate_tables(session):
    """
    loop through all the rows in the combined CSV, `final_csv`, and populate tables with their data

    Note: not all fields in the CSV are currently used
    """
    start = time.perf_counter()
    with open(final_csv, encoding='utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader)  # skip header
        for line in csvreader:
            artist_name = line[0]
            first_name = line[1]
            gender = line[2]
            item_in_session = int(line[3])
            last_name = line[4]
            length = float(line[5])
            level = line[6]
            location = line[7]
            session_id = int(line[8])
            song = line[9]
            user_id = int(line[10])
            session.execute(songs_by_session.insert, (session_id, item_in_session, artist_name, song, length))
            session.execute(songs_by_player.insert,
                            (user_id, session_id, item_in_session, artist_name, song, f"{first_name} {last_name}"))
            session.execute(players_by_song.insert, (song, user_id, f"{first_name} {last_name}"))

        stop = time.perf_counter()
        print(f"inserts in {stop - start:0.4f} seconds")

In [9]:
def select_tables(session):
    """
    Run initial given queries and store results
    """
    songs_by_session_df = pd.DataFrame(list(session.execute(songs_by_session.select)))
    songs_by_player_df = pd.DataFrame(list(session.execute(songs_by_player.select)))
    players_by_song_df = pd.DataFrame(list(session.execute(players_by_song.select)))

    return {'query1': songs_by_session_df,
            'query2': songs_by_player_df,
            'query3': players_by_song_df}

In [10]:
cluster = None
session = None
results = {}

filepaths = create_filepaths()
create_final_csv(filepaths)

try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()

    recreate_keyspace(session)
    use_keyspace(session)
    drop_tables(session)
    create_tables(session)
    populate_tables(session)
    results = select_tables(session)

except Exception as e:
    print(f"Exception:\n{e}")
finally:
    session.shutdown()
    cluster.shutdown()

drop_tables in 0.3741 seconds
create_tables in 0.1473 seconds
inserts in 8.6432 seconds


# Results

## Query 1:
Answers the question:
> Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

It uses the following table & SQL query to accomplish this:

In [11]:
print(songs_by_session.select)


SELECT artist_name, song, length
  FROM songs_by_session
 WHERE session_id = 338
   AND item_in_session = 4



The results are stored in `query1`, printed below in the output:

In [12]:
results['query1'].style.hide_index()

/tmp/ipykernel_27151/469587350.py:1: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  results['query1'].style.hide_index()


artist_name,song,length
Faithless,Music Matters (Mark Knight Dub),495.307300


## Query 2:
Ansers the question:
> Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

It uses the following table & SQL query to accomplish this:

In [13]:
print(songs_by_player.select)


SELECT artist_name, song, user
  FROM songs_by_player
 WHERE user_id = 10
   AND session_id = 182



The results are stored in `query2`, printed below in the output:

In [14]:
results['query2']

,artist_name,song,user
0,Down To The Bone,Keep On Keepin' On,Sylvie Cruz
1,Three Drives,Greece 2000,Sylvie Cruz
2,Sebastien Tellier,Kilometer,Sylvie Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie Cruz


## Query 3:
Answers the question:
> Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

It uses the following table & SQL query to accomplish this:

In [15]:
print(players_by_song.select)


SELECT user
  FROM players_by_song
 WHERE song = 'All Hands Against His Own'



The results are stored in `query3`, printed below in the output:

In [16]:
results['query3']

,user
0,Jacqueline Lynch
1,Tegan Levine
2,Sara Johnson
